In [1]:
# !pip install -U datasets

In [2]:
# !sudo -H pip install -q transformers --upgrade

In [5]:
# !pip uninstall autoawq -y

In [ ]:
# !pip install autoawq

In [3]:
# !sudo -H pip install https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.7/autoawq-0.1.7+cu118-cp38-cp38-linux_x86_64.whl

In [4]:
# !pip install transformers==4.35.2

In [1]:
import pandas
import numpy
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer,AutoModelForCausalLM
import torch
quant_path = '/llmmodels/quantized_model/awq_mistral'
model_path='mistralai/Mistral-7B-v0.1'

In [2]:
quant_config = {"zero_point":True,
               "q_group_size": 128,
               "w_bit": 4,
               "version": "GEMM"
               }

In [8]:
#load model
# model = AutoAWQForCausalLM.from_pretrained(model_path, **{"low_cpu_mem_usage":True})
model = AutoAWQForCausalLM.from_pretrained(model_path, device_map='auto')

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

In [10]:
import torch
torch.cuda.empty_cache()

In [11]:
import gc
gc.collect()


31

In [ ]:
# torch.cuda.memory_summary(device=None, abbreviated=False)

In [12]:
#Quantize
# dont run it again
model.quantize(tokenizer, quant_config=quant_config)

/tmp/pip_packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
AWQ: 100%|██████████| 32/32 [41:02<00:00, 76.96s/it]


In [ ]:
# model_quantized = AutoModelForCausalLM.from_pretrained(
#     model,
#     torch_dtype=torch.float16,
#     device_map="auto",
#     quantization_config=quant_config
# )

In [13]:
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

('/llmmodels/quantized_model/awq_mistral/tokenizer_config.json',
 '/llmmodels/quantized_model/awq_mistral/special_tokens_map.json',
 '/llmmodels/quantized_model/awq_mistral/tokenizer.model',
 '/llmmodels/quantized_model/awq_mistral/added_tokens.json',
 '/llmmodels/quantized_model/awq_mistral/tokenizer.json')

### Inference on quantized model

In [4]:
from transformers import pipeline
quant_path = '/llmmodels/quantized_model/awq_mistral'
model_path='mistralai/Mistral-7B-v0.1'

In [10]:
model_id = quant_path
pipe = pipeline("text-generation", model=model_id, device_map="auto")

In [17]:
import time
start = time.time()
output2 = pipe("I dont know my name",
               max_new_tokens=500,
               num_return_sequences=1,
               do_sample=True,
               top_k=50,
               top_p=0.95)[0]["generated_text"]
print("time taken is :", time.time()-start)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


time taken is : 21.161255836486816


In [18]:
print(output2)

I dont know my name." It was like... "Hey, I'm sorry man, but I've gotta go do a thing now. Maybe I'll see you at the show."

Congrats to all of the WILDERUN staff who made it to our second gig ever and I hope that you'll be a part of the next, whenever that is.

*  *  *  *  *

Anyway, our opening act was this little band called HUNTRESS, whom you may have heard of. We weren't necessarily too familiar with them before we met them, but it turns out that I'd heard their cover of MORBID ANGEL'S "Rapture" on one of the many metal podcasts that I listen to and absolutely loved it. The guitar solo is phenomenal, and I can never quite decide if it's better than the original or not. If you haven't heard it, I highly recommend that you check it out, if only to hear some of the best growls in heavy metal. It's certainly up there with HATEBREED's cover of CRUNCHY BREAKDOWN.

So, I go backstage to say hello, and I'm greeted by Jill Janus. She's not tall, so maybe she's shorter than me, but she's d

### Inference on un-quantized model

In [3]:
from transformers import pipeline

In [5]:
model_id = model_path
pipe = pipeline("text-generation", model=model_id, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
# Using unquant model
import time
start = time.time()
output2 = pipe("I dont know my name",
               max_new_tokens=500,
               num_return_sequences=1,
               do_sample=True,
               top_k=50,
               top_p=0.95)[0]["generated_text"]
print("time taken is :", time.time()-start)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


time taken is : 31.65123462677002


In [11]:
print(output2)

I dont know my name I dont know my name.

Jeremiah 6:16
16 This is what the LORD says: "Stand at the crossroads and look; ask for the ancient paths, ask where the good way is, and walk in it, and you will find rest for your souls. But you said, 'We will not walk in it.'"

Hosea 6:6
6 I desire mercy, not sacrifice, and acknowledgement of God rather than burnt offerings.

Matthew 23:23
23 "Woe to you, teachers of the law and Pharisees, you hypocrites! You give a tenth of your spices - mint, dill and cumin. But you have neglected the more important matters of the law - justice, mercy and faithfulness. You should have practiced the latter, without neglecting the former.

James 1:27
27 Religion that God our Father accepts as pure and faultless is this: to look after orphans and widows in their distress and to keep oneself from being polluted by the world.

Mark 1:14-15
14 After John was put in prison, Jesus went into Galilee, proclaiming the good news of God. 15 "The time has come," he said

In [ ]:
model

In [ ]:
quant_model